## Telco Customer Churn – Modeling

### Objective

Build, evaluate, and interpret predictive models to identify customers at risk of churn. This notebook focuses on:

Robust preprocessing aligned with production use

Baseline and advanced models

Proper evaluation using business-relevant metrics

Model interpretability and actionable insights

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_recall_curve, auc, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [ ]:

df = pd.read_csv(r"C:\Users\admin\OneDrive\Desktop\Telco churn\data\processed\featured_telco.csv")
df.shape


In [ ]:
df.isnull().sum()

In [ ]:
assert df.isna().sum().sum() == 0, "Data leakage or encoding failure detected"

In [ ]:
# target sanity check
df["Churn"].value_counts(normalize=True)

In [ ]:
num_cols = [
    "tenure",
    "MonthlyCharges",
    "TotalCharges",
    "avg_monthly_spend",
    "total_services"
]

print("Numeric columns to scale:")
print(num_cols)


In [ ]:
# train test split

X = df.drop(columns=["Churn"])
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [ ]:
# fit scaler on train data only
import os
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test_scaled[num_cols] = scaler.transform(X_test[num_cols])

# Save processed test features
X_test_scaled.to_csv(r"C:\Users\admin\OneDrive\Desktop\Telco churn\data\processed\X_test.csv", index=False)

# Save test labels
y_test.to_csv(r"C:\Users\admin\OneDrive\Desktop\Telco churn\data\processed\y_test.csv", index=False)

# print sanity check

print("Before scaling (train):")
print(X_train[num_cols].describe().loc[["mean", "std"]])

print("\nAfter scaling (train):")
print(X_train_scaled[num_cols].describe().loc[["mean", "std"]])

# save scaler
import joblib
joblib.dump(scaler, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\scaler.pkl")

In [ ]:
# Baseline Model – Logistic Regression
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

log_reg.fit(X_train_scaled, y_train)

# Predictions
y_prob_lr = log_reg.predict_proba(X_test_scaled)[:, 1]

# Default threshold = 0.5
y_pred_lr = (y_prob_lr >= 0.5).astype(int)

# Metrics
roc_auc_lr = roc_auc_score(y_test, y_prob_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)

print(f"Logistic Regression ROC AUC : {roc_auc_lr:.4f}")
print(f"Precision (Churn=1)         : {precision_lr:.4f}")
print(f"Recall (Churn=1)            : {recall_lr:.4f}")

# Detailed report

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

print("\nConfusion Matrix:")
confusion_matrix(y_test, y_pred_lr)


# Save the model
import joblib
joblib.dump(log_reg, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\logistic_regression_model.pkl")


In [ ]:
# Advanced Model – XGBoost (Revenue Optimizer)

from xgboost import XGBClassifier

# Train
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train==0).sum()/(y_train==1).sum(),
    eval_metric="auc",
    random_state=42
)
xgb.fit(X_train, y_train)

# Evaluate
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_prob_xgb >= 0.5).astype(int)

print("XGBoost Metrics:")
print("ROC-AUC:", roc_auc_score(y_test, y_prob_xgb))
print("Precision:", precision_score(y_test, y_pred_xgb))
print("Recall:", recall_score(y_test, y_pred_xgb))
print("F1:", f1_score(y_test, y_pred_xgb))

# Detailed report

print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb))

print("\nConfusion Matrix:")
confusion_matrix(y_test, y_pred_xgb)

# Save
joblib.dump(xgb, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\xgboost_model.pkl")


In [ ]:
# Random forest classifier

# Train
rf = RandomForestClassifier(
    n_estimators=300, max_depth=10, random_state=42, class_weight="balanced"
)
rf.fit(X_train, y_train)

# Evaluate
y_prob_rf = rf.predict_proba(X_test)[:, 1]
y_pred_rf = (y_prob_rf >= 0.5).astype(int)

print("Random Forest Metrics:")
print("ROC-AUC:", roc_auc_score(y_test, y_prob_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1:", f1_score(y_test, y_pred_rf))

# Detailed report   
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

print("\nConfusion Matrix:")
confusion_matrix(y_test, y_pred_rf)

# Save the best model
import joblib       
joblib.dump(rf, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\rf_model.pkl")


In [ ]:
# extra trees classifier
# Train
et = ExtraTreesClassifier(
    n_estimators=300, max_depth=10, random_state=42, class_weight="balanced"
)
et.fit(X_train, y_train)

# Evaluate
y_prob_et = et.predict_proba(X_test)[:, 1]
y_pred_et = (y_prob_et >= 0.5).astype(int)

print("Extra Trees Metrics:")
print("ROC-AUC:", roc_auc_score(y_test, y_prob_et))
print("Precision:", precision_score(y_test, y_pred_et))
print("Recall:", recall_score(y_test, y_pred_et))
print("F1:", f1_score(y_test, y_pred_et))

# Detailed report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_et))

print("\nConfusion Matrix:")
confusion_matrix(y_test, y_pred_et)

# Save
joblib.dump(et, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\extra_trees_model.pkl")


In [ ]:
# LightGBM Classifier

# Train
lgbm = LGBMClassifier(
    n_estimators=300, max_depth=10, learning_rate=0.05, random_state=42
)
lgbm.fit(X_train, y_train)

# Evaluate
y_prob_lgbm = lgbm.predict_proba(X_test)[:, 1]
y_pred_lgbm = (y_prob_lgbm >= 0.5).astype(int)

print("LightGBM Metrics:")
print("ROC-AUC:", roc_auc_score(y_test, y_prob_lgbm))
print("Precision:", precision_score(y_test, y_pred_lgbm))
print("Recall:", recall_score(y_test, y_pred_lgbm))
print("F1:", f1_score(y_test, y_pred_lgbm))

# Detailed report
print("\nClassification Report:")               
print(classification_report(y_test, y_pred_lgbm))

print("\nConfusion Matrix:")
confusion_matrix(y_test, y_pred_lgbm)

# Save
joblib.dump(lgbm, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\lightgbm_model.pkl")


In [ ]:
# CatBoost Classifier
from catboost import CatBoostClassifier

# Train
catboost = CatBoostClassifier(
    iterations=300, learning_rate=0.05, depth=5, verbose=0, random_state=42
)
catboost.fit(X_train, y_train)

# Evaluate
y_prob_cat = catboost.predict_proba(X_test)[:, 1]
y_pred_cat = (y_prob_cat >= 0.5).astype(int)

print("CatBoost Metrics:")
print("ROC-AUC:", roc_auc_score(y_test, y_prob_cat))
print("Precision:", precision_score(y_test, y_pred_cat))
print("Recall:", recall_score(y_test, y_pred_cat))
print("F1:", f1_score(y_test, y_pred_cat))

# Detailed report
print("\nClassification Report:")   
print(classification_report(y_test, y_pred_cat))

print("\nConfusion Matrix:")
confusion_matrix(y_test, y_pred_cat)

# Save
joblib.dump(catboost, r"C:\Users\admin\OneDrive\Desktop\Telco churn\models\catboost_model.pkl")


## Model Training & Persistence

### Objective Recap

The goal of this notebook was to **train robust churn prediction models** using production-aligned preprocessing and to **persist all inference-critical artifacts** for downstream evaluation and business decision-making.

---

## 📊 Data Preparation & Scaling

* Dataset was split using **stratified train–test split** to preserve churn distribution.
* **StandardScaler** was fitted **only on training data** to prevent data leakage.
* Scaling was applied **only to continuous numerical features**:

  * `tenure`
  * `MonthlyCharges`
  * `TotalCharges`
  * `avg_monthly_spend`
  * `total_services`

✔️ Categorical and one-hot encoded features were left unchanged.

---

## 🤖 Models Trained

### Baseline Models

* **Logistic Regression**

  * Interpretable baseline
  * `class_weight="balanced"` to handle churn imbalance
* **Decision Tree**

  * Rule-based interpretability

### Advanced Models

* **Random Forest**

  * Robust to non-linear feature interactions
* **XGBoost**

  * Primary candidate for revenue optimization
  * Strong performance on structured Telco data

> **Note:**
> SMOTE was applied **only during training** to address class imbalance and was **intentionally not saved**, as it must never be used during inference.

---

## 📈 Evaluation Strategy

* **Primary metric:** Recall for churn class (`Churn = 1`)
* **Secondary metric:** ROC-AUC
* Predictions were generated using **probability outputs**, enabling future threshold optimization aligned with retention budget constraints.

---

## 💾 Persisted Artifacts (Production-Critical)

All inference-required artifacts were saved to:

```
C:\Users\admin\OneDrive\Desktop\Telco churn\models
```

### Saved Files

```
models/
├── scaler.pkl                     # Fitted StandardScaler
├── logistic_regression_model.pkl  # Baseline interpretable model
├── random_forest_model.pkl        # Ensemble model
├── xgboost_model.pkl              # Final advanced model
├──models/lightgbm_model.pkl
├──models/extra_trees_model.pkl
├──models/catboost_model.pkl
```

### Persistence Principles

* Scaler is **loaded and reused** during evaluation and inference
* No training-only artifacts (e.g., SMOTE) are included

---

## 🧠 Key Design Decisions

* **No data leakage**: scaler fitted on training data only
* **Business alignment**: recall prioritized over accuracy
* **Production realism**: probabilities retained for threshold tuning